In [7]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Qdrant

In [8]:
# Specify the dataset name and the column containing the content
ds1 = "flytech/python-codes-25k"
page_content_column = "text"  # or any other column you're interested in


# Specify the dataset name and the column containing the content
ds2 = "AlaaElhilo/Wikipedia_ComputerScience"
page_content_column2 = "Text"  # or any other column you're interested in

In [9]:
# Create an instance of the HuggingFaceDatasetLoader
loader1 = HuggingFaceDatasetLoader(ds1, page_content_column)
docs1 = loader1.load()

loader2 = HuggingFaceDatasetLoader(ds2, page_content_column2)
docs2 = loader2.load()

# Combine the documents from both datasets
docs = docs1 + docs2

/workspaces/RAG_Bot/.venv/lib/python3.10/site-packages/datasets/load.py:2554: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


In [10]:
# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

# 'data' holds the text you want to split, split the text into documents using the text splitter.
doc = text_splitter.split_documents(docs)

In [11]:
# Define the path to the pre-trained model you want to use
modelPath1 = "sentence-transformers/all-MiniLM-L12-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs1 = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs1 = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath1,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs1, # Pass the model configuration options
    encode_kwargs=encode_kwargs1 # Pass the encoding options
)

In [14]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    path="Qdrant_VDB",
    collection_name="CS_doc",
)

RuntimeError: Storage folder Qdrant_VDB is already accessed by another instance of Qdrant client. If you require concurrent access, use Qdrant server instead.

In [13]:
query = "What is java?"
found_docs = qdrant.similarity_search(query)

print(found_docs[0].page_content)

"Due to the rapid growth of the Internet and the World Wide Web in the 1990s, new programming languages were introduced to support Web pages and networking. Java, based on C++ and designed for increased portability across systems and security, enjoyed large-scale success because these features are essential for many Internet applications. Another development was that of dynamically typed scripting languages\u2014Python, JavaScript, PHP, and Ruby\u2014designed to quickly produce small programs that coordinate existing applications. Due to their integration with HTML, they have also been used for building web pages hosted on servers."
